In [1]:
%%time

import numpy as np
import torch
import gc
import random
from torch import nn
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm
from NN_models import NN_2_256, NN_4_256, NN_8_256, NN_8_64, test # NN_8_128
from reaction_energy_calculation import calculate_reaction_energy
from prepare_data import prepare, save_chk, load_chk


def set_random_seed(seed):
    # seed everything
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(41)

CPU times: user 839 ms, sys: 270 ms, total: 1.11 s
Wall time: 1.4 s


In [2]:
# %%time
# data, data_train, data_test = prepare(path='data', test_size=0.2)

CPU times: user 33.9 s, sys: 13.2 s, total: 47.1 s
Wall time: 1min 7s


In [10]:
# %%time
# save_chk(data, data_train, data_test, path='checkpoints')

CPU times: user 2.58 s, sys: 9.61 s, total: 12.2 s
Wall time: 13.3 s


In [11]:
%%time
data, data_train, data_test = load_chk(path='checkpoints')

CPU times: user 4.14 s, sys: 3.77 s, total: 7.91 s
Wall time: 7.94 s


In [3]:
data[73] # take the easiest reaction H2 = 2H

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype='<U20'),
 'Coefficients': tensor([ 2., -1.]),
 'Energy': tensor(109.4900),
 'Grid': tensor([[ -1.1678, -16.1181, -16.0818,  ..., -16.1181, -16.1033, -16.1181],
         [ -1.1678, -16.1181, -14.7017,  ..., -16.1181, -15.3051, -16.1181],
         [ -1.1678, -16.1181, -12.3610,  ..., -16.1181, -13.2385, -16.1181],
         ...,
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148],
         [ -7.7005,  -7.7005, -13.7295,  ..., -13.7295,  -8.2041,  -8.2041],
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02]),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.0000],
         ...,
         [0.0003, 0.0003],
         [0.0005, 0.0005],
         [0.0003, 0.0003]]),
 'Gradients': tensor([[3.6975e-09, 0.0000e+00, 0.0000e+00],

In [9]:
data_test[0]

{'Database': 'NCCE31',
 'Components': array(['HCl-HCl_ncce31', 'HCl_ncce31'], dtype='<U20'),
 'Coefficients': tensor([-1.,  2.]),
 'Energy': tensor(2.0100),
 'Grid': tensor([[1.8193, 1.8079, 1.9052,  ..., 1.9413, 1.8151, 1.8091],
         [1.8193, 1.8079, 2.0157,  ..., 2.0501, 1.8209, 1.8145],
         [1.8193, 1.8078, 2.0798,  ..., 2.1132, 1.8580, 1.8490],
         ...,
         [1.1998, 1.2391, 1.2329,  ..., 1.2795, 1.1805, 1.2183],
         [1.2115, 1.2498, 1.2455,  ..., 1.2918, 1.1918, 1.2289],
         [1.1998, 1.2391, 1.2329,  ..., 1.2795, 1.1805, 1.2183]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.5979e-02, 2.2624e-02,
         2.5979e-02]),
 'Densities': tensor([[1.5987e+03, 1.5987e+03],
         [1.5986e+03, 1.5986e+03],
         [1.5977e+03, 1.5977e+03],
         ...,
         [2.7319e-04, 2.7319e-04],
         [3.6636e-04, 3.6636e-04],
         [2.7319e-04, 2.7319e-04]]),
 'Gradients': tensor([[2.1993e+05, 8.7972e+05, 2.1993e+05],
         [1.8339e+07,

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        # i = 1
        self.data[i].pop('Database', None)

        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())


train_set = Dataset(data=data_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                               batch_size=None,
                                               num_workers=1,
                                               pin_memory=True,
                                               shuffle=True)

test_set = Dataset(data=data_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                              batch_size=None,
                                              num_workers=1,
                                              pin_memory=True,
                                              shuffle=True)

In [12]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
print(device)
# model = NN_2_256(nconstants=21, DFT='SVWN').to(device)

model = NN_4_256(nconstants=24, DFT='PBE').to(device)

# model.load_state_dict(torch.load('predopt/PBE_2_256.param', map_location=device))

cuda:0


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

In [284]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3)

In [249]:
import torch_optimizer as optim2


optimizer = optim2.RAdam(
    model.parameters(),
    lr= 3e-4,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
)

In [241]:
optimizer = optim2.QHAdam(
    model.parameters(),
    lr= 1e-4,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,
)

In [13]:
criterion = nn.MSELoss()

In [233]:
from importlib import reload
import predopt
reload(predopt)

<module 'predopt' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/predopt.py'>

In [282]:
%%time

from predopt import DatasetPredopt
train_predopt_set = DatasetPredopt(data=data, dft='PBE')
train_predopt_dataloader = torch.utils.data.DataLoader(train_predopt_set,
                                                       batch_size=None,
                                                       num_workers=1,
                                                       pin_memory=True,
                                                       shuffle=True)

CPU times: user 188 µs, sys: 0 ns, total: 188 µs
Wall time: 197 µs


In [283]:
from predopt import predopt


predopt(model, criterion, optimizer, train_predopt_dataloader, device, n_epochs=4, accum_iter=10)

Epoch 1


  0%|          | 0/267 [00:00<?, ?it/s]

train MSE Loss = 150.11520275
train MAE Loss = 3.71139860
Epoch 2


  0%|          | 0/267 [00:00<?, ?it/s]

train MSE Loss = 1.59235512
train MAE Loss = 0.87656474
Epoch 3


  0%|          | 0/267 [00:00<?, ?it/s]

train MSE Loss = 1.27286607
train MAE Loss = 0.78368872
Epoch 4


  0%|          | 0/267 [00:00<?, ?it/s]

train MSE Loss = 1.38013759
train MAE Loss = 0.80799741


([150.115202749217,
  1.5923551232726485,
  1.2728660724781178,
  1.3801375936578821],
 [3.7113986, 0.87656474, 0.7836887, 0.8079974])

In [256]:
# torch.save(model.state_dict(), 'predopt/PBE_4_256.param')

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))

In [13]:
torch.cuda.empty_cache()

In [263]:
from tqdm.notebook import tqdm
import os

log_file_path = 'log/epoch_training.log'

if os.path.isfile(log_file_path):
    os.remove(log_file_path)

In [269]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20, accum_iter=4, verbose=False):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        model.train()
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for batch_idx, (X_batch, y_batch) in enumerate(progress_bar_train):
            
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = model(X_batch_grid)
            
            # print(predictions)
            # predictions = torch.tile(torch.Tensor(true_constants_PBE), [X_batch_grid.shape[0],1]).to(device, non_blocking=True)
            # print(predictions)
            # print(torch.max(torch.abs(predictions)))
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f}")
            loss = criterion(reaction_energy, y_batch)

            # loss_accumulation
            loss = loss / accum_iter
            
            loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)

                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                train_mse_losses_per_epoch.append(MSE)
                train_mae_losses_per_epoch.append(MAE)
                progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)

                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                    # f.write(f"{preds}\n")
                    
                del MAE, MSE
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy
            gc.collect()
            torch.cuda.empty_cache()
            # gpu_usage()
        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        progress_bar_test = tqdm(test_loader)
        model.eval()
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in progress_bar_test:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                # print(f"{X_batch['Components']}")
                predictions = model(X_batch_grid)
                reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
                loss = criterion(reaction_energy, y_batch)
                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                progress_bar_train.set_postfix(MAE = MAE)
                test_mse_losses_per_epoch.append(MSE)
                test_mae_losses_per_epoch.append(MAE)
                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                
                del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
                gc.collect()
                torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [270]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [273]:
train_loss_mae = train(model, criterion, optimizer,
                              train_dataloader,
                              test_dataloader, n_epochs=1)

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

res_fH is NaN
res_fH is inf
log is NaN
log is inf


ValueError: NaN detected

In [ ]:
train_loss_mae

In [14]:
def test_dft(train_loader, test_loader, c_arr, criterion, rung, dft, verbose=False, n_epochs=1):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_train:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            # print(torch.tile(c_arr, [X_batch_grid.shape[0],1]))
            predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")

            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()

        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        progress_bar_test = tqdm(test_loader)
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_test:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = torch.tile(c_arr, (X_batch_grid.shape[0],1)).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            # progress_bar_train.set_postfix(MAE = MAE)
            test_mse_losses_per_epoch.append(MSE)
            test_mae_losses_per_epoch.append(MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [18]:
from importlib import reload
import NN_models
reload(NN_models)
from NN_models import NN_2_256, NN_8_256, NN_8_64
# import SVWN3

In [19]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [20]:
true_constants_SVWN = torch.Tensor([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*torch.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])

true_constants_PBE = torch.Tensor([[0.06672455,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709921,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 3.3662,  0.88026,
       0.49294, 0.62517, 0.49671,
       # 1,  1,  1,
       0.031091, 0.015545, 0.016887,
       0.21370,  0.20548,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])

par_rpw92_constants = torch.Tensor([[0.06672455,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709920934161365617563962776245,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 4.86059,  0.88026,
       0.49294, 0.750188, 0.49671,
       # 1,  1,  1,
       0.0310907, 0.01554535, 0.0168869,
       0.21370,  0.266529,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])


# true_constants

test_dft(train_dataloader, test_dataloader, true_constants_PBE, criterion, rung='GGA', dft='PBE', verbose=False)
# test_dft(train_dataloader, test_dataloader, true_constants_SVWN, criterion, rung='LDA', dft='SVWN3', verbose=False)

# broken_PBE 45.47375717 35.76542641
# Mean Absolute Error, kcal/mol
# DFT              train        test
# SVWN3            32.63222432 26.81526929
# PBE_sigma_broken 7.84055060  7.63013082
# PBE_fixed        7.85833597  7.64929252

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

train MSE Loss = 115.08597731 MAE Loss = 7.85833597


  0%|          | 0/52 [00:00<?, ?it/s]

test MSE Loss = 108.85051548 MAE Loss = 7.64929252


[7.858335966790138]

In [16]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)
from reaction_energy_calculation import calculate_reaction_energy

In [17]:
from importlib import reload
import PBE
reload(PBE)
from PBE_new import F_PBE_new

In [23]:
calculate_reaction_energy(data[73], torch.tile(true_constants_PBE, [data[73]['Grid'].shape[0],1]).to(device='cpu'), device='cpu', rung='GGA', dft='PBE')

tensor(104.6450)